In [1]:
import numpy as np
import _pickle as pickle
import gym
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.initializers as initializers
import random

In [2]:
class Memory:
    def __init__(self, max_memory):
        self.max_memory = max_memory
        self.samples = []
    
    def add_sample(self, sample):
        self.samples.append(sample)
        if len(self.samples) > self.max_memory:
            self.samples.pop(0)
            print("What's Poppin")
            
    def sample(self, no_samples):
        if no_samples > len(self.samples):
            return random.sample(self.samples, len(self.samples))
        else:
            return random.sample(self.samples, no_samples)

In [3]:
initializer = initializers.GlorotNormal

x_in = layers.Input(shape = (6400,))
x = layers.Dense(200, kernel_initializer= initializer, activation="relu")(x_in)
x_out = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(x_in, x_out) # Use this for fitting as epsilon will bring it all over the damm place

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

In [4]:
target_model = model # Use this one for prediction to have some semblence of consistency

target_model.set_weights(model.get_weights())

In [5]:
class GameRunner:
    def __init__(self, env, model, target_model, memory, epsilon, max_eps, min_eps, game_dimensions, epsilon_greedy_frames, resume = False, render = True):
        self.env = env
        self.model = model
        self.target_model = target_model
        self.memory = memory
        self.eps = epsilon
        self.max_eps = max_eps
        self.min_eps = min_eps
        self.render = render
        self.resume = resume
        self.epsilon_greedy_frames = epsilon_greedy_frames
        self.gameDimensions = game_dimensions
        self.rewards = []
        self.max_x = []
    
    def run(self):
        observation = self.env.reset()
        reward_sum = 0
        running_reward = None
        prev_frame = None
        episode_number = 0
        
        while True:
            if self.render:
                env.render()

            if self.resume:
                self.model.load_weights("ModelWeights")
            
            curr_frame = self.prepro(observation)
            change_in_frame = curr_frame - prev_frame if prev_frame is not None else np.zeros(self.gameDimensions)
            prev_frame = curr_frame
            
            action, up_prob = self.choose_action(curr_frame)
            
            observation, reward, done, _ = self.env.step(action) 
                
            y = 1 if action == 2 else 0
                        
            self.memory.add_sample((change_in_frame, y - up_prob, reward, done))
            
            # Decay probability of taking random action
            epsilon_interval = (self.max_eps - self.min_eps)
            self.eps -= epsilon_interval / self.epsilon_greedy_frames
            self.eps = max(self.eps, self.min_eps)
            
            reward_sum += reward
            
            if done:
                if episode_number % 10 == 0: # Probably should adjust this number
                    self.target_model.set_weights(self.model.get_weights())
                    self.target_model.save_weights("ModelWeights")
                
                self.replay(self.memory.samples)
                
                self.memory.samples = []
                
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print ('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
                reward_sum = 0
                observation = env.reset() # reset env
                prev_frame = None
                episode_number += 1

            if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
                print('ep %d: game finished, reward: %f, epsilon: %g' % (episode_number, reward, self.eps) + ('' if reward == -1 else ' !!!!!!!!'))
                
    def choose_action(self, state):
        if np.random.random() < self.eps:
            randn = np.random.randint(2, 4)
            up_prob = 0
            return np.random.randint(2, 4), up_prob
        else:
            state = state.reshape((1,6400))
            up_prob = self.target_model.predict(state)
            if up_prob >= .5:
                return 2, up_prob
            else:
                return 3, up_prob
    
    def discount_rewards(self, rewards):
        """ take 1D float array of rewards and compute discounted reward """
        gamma = 0.99
        discounted_r = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
            running_add = running_add * gamma + rewards[t]
            discounted_r[t] = running_add
        return discounted_r
            
    def prepro(self, input_frame):
        """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
        input_frame = input_frame[34:194] # crop
        input_frame = input_frame[::2,::2,0] # downsample by factor of 2 (halves the resolution of the image)
        #This takes every other pixel in the image
        input_frame[input_frame == 144] = 0 # erase background (background type 1)
        input_frame[input_frame == 109] = 0 # erase background (background type 2)
        input_frame[input_frame != 0] = 1 # everything else (paddles, ball) just set to 1
        return input_frame.astype(np.float).ravel()
    
    def replay(self, samples):
        states = np.array([val[0] for val in samples])
        up_probs = np.array([val[1] for val in samples])
        rewards = np.array([val[2] for val in samples])
        
        up_probs = np.vstack(up_probs)
        rewards = np.vstack(rewards)
        
        q_s_a = self.target_model.predict(states)
        
        discount_rewards = self.discount_rewards(rewards)
        discount_rewards -= np.mean(discount_rewards)
        discount_rewards /= np.std(discount_rewards)
        
        up_probs = up_probs * discount_rewards
        
        up_probs = np.asarray(up_probs).astype('float32')
        
        self.model.fit(states, up_probs, batch_size = len(samples), verbose = 1, epochs = 50)

In [6]:
env = gym.make("Pong-v0")

mem = Memory(100_000)

eps = 1.0
max_eps = 1.0
min_eps = 0.000001
eps_greedy_frames = 100000.0

game_dimensions = 80*80

gr = GameRunner(env, model, target_model, mem, eps, max_eps, min_eps, game_dimensions, eps_greedy_frames, resume = False, render = True)

gr.run()

ep 0: game finished, reward: -1.000000, epsilon: 0.99911
ep 0: game finished, reward: -1.000000, epsilon: 0.99867
ep 0: game finished, reward: -1.000000, epsilon: 0.99744
ep 0: game finished, reward: -1.000000, epsilon: 0.99692
ep 0: game finished, reward: -1.000000, epsilon: 0.99645
ep 0: game finished, reward: -1.000000, epsilon: 0.99601
ep 0: game finished, reward: -1.000000, epsilon: 0.99555
ep 0: game finished, reward: -1.000000, epsilon: 0.99431
ep 0: game finished, reward: -1.000000, epsilon: 0.99382
ep 0: game finished, reward: -1.000000, epsilon: 0.99333
ep 0: game finished, reward: -1.000000, epsilon: 0.99287
ep 0: game finished, reward: -1.000000, epsilon: 0.99244
ep 0: game finished, reward: -1.000000, epsilon: 0.99198
ep 0: game finished, reward: -1.000000, epsilon: 0.99153
ep 0: game finished, reward: -1.000000, epsilon: 0.99108
ep 0: game finished, reward: -1.000000, epsilon: 0.99062
ep 0: game finished, reward: -1.000000, epsilon: 0.99015
ep 0: game finished, reward: -1

<ipython-input-5-4a3d2e52a5b4>:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  up_probs = np.array([val[1] for val in samples])


Epoch 1/50
1/1 [==============================] - 0s 388ms/step - loss: 0.6923 - acc: 0.2406
Epoch 2/50
1/1 [==============================] - 0s 42ms/step - loss: 0.6435 - acc: 0.4215
Epoch 3/50
1/1 [==============================] - 0s 49ms/step - loss: 0.5983 - acc: 0.4735
Epoch 4/50
1/1 [==============================] - 0s 54ms/step - loss: 0.5568 - acc: 0.4821
Epoch 5/50
1/1 [==============================] - 0s 48ms/step - loss: 0.5186 - acc: 0.4829
Epoch 6/50
1/1 [==============================] - 0s 50ms/step - loss: 0.4833 - acc: 0.4846
Epoch 7/50
1/1 [==============================] - 0s 47ms/step - loss: 0.4502 - acc: 0.4846
Epoch 8/50
1/1 [==============================] - 0s 46ms/step - loss: 0.4192 - acc: 0.4846
Epoch 9/50
1/1 [==============================] - 0s 59ms/step - loss: 0.3896 - acc: 0.4846
Epoch 10/50
1/1 [==============================] - 0s 61ms/step - loss: 0.3613 - acc: 0.4846
Epoch 11/50
1/1 [==============================] - 0s 64ms/step - loss: 0.3342

1/1 [==============================] - 0s 43ms/step - loss: -0.7455 - acc: 0.4902
Epoch 25/50
1/1 [==============================] - 0s 51ms/step - loss: -0.7777 - acc: 0.4894
Epoch 26/50
1/1 [==============================] - 0s 54ms/step - loss: -0.8100 - acc: 0.4902
Epoch 27/50
1/1 [==============================] - 0s 43ms/step - loss: -0.8427 - acc: 0.4902
Epoch 28/50
1/1 [==============================] - 0s 43ms/step - loss: -0.8757 - acc: 0.4902
Epoch 29/50
1/1 [==============================] - 0s 45ms/step - loss: -0.9090 - acc: 0.4902
Epoch 30/50
1/1 [==============================] - 0s 47ms/step - loss: -0.9426 - acc: 0.4902
Epoch 31/50
1/1 [==============================] - 0s 47ms/step - loss: -0.9766 - acc: 0.4911
Epoch 32/50
1/1 [==============================] - 0s 48ms/step - loss: -1.0110 - acc: 0.4911
Epoch 33/50
1/1 [==============================] - 0s 43ms/step - loss: -1.0458 - acc: 0.4911
Epoch 34/50
1/1 [==============================] - 0s 42ms/step - loss: 

1/1 [==============================] - 0s 50ms/step - loss: -2.1383 - acc: 0.4803
Epoch 48/50
1/1 [==============================] - 0s 55ms/step - loss: -2.1862 - acc: 0.4803
Epoch 49/50
1/1 [==============================] - 0s 52ms/step - loss: -2.2345 - acc: 0.4803
Epoch 50/50
1/1 [==============================] - 0s 51ms/step - loss: -2.2834 - acc: 0.4803
resetting env. episode reward total was -21.000000. running mean: -21.000000
ep 3: game finished, reward: -1.000000, epsilon: 0.96231
ep 3: game finished, reward: -1.000000, epsilon: 0.96151
ep 3: game finished, reward: -1.000000, epsilon: 0.96107
ep 3: game finished, reward: -1.000000, epsilon: 0.96063
ep 3: game finished, reward: -1.000000, epsilon: 0.96017
ep 3: game finished, reward: -1.000000, epsilon: 0.95968
ep 3: game finished, reward: -1.000000, epsilon: 0.95922
ep 3: game finished, reward: -1.000000, epsilon: 0.95878
ep 3: game finished, reward: -1.000000, epsilon: 0.95835
ep 3: game finished, reward: -1.000000, epsilo

...

1/1 [==============================] - 0s 41ms/step - loss: -0.2276 - acc: 0.8585
Epoch 35/50
1/1 [==============================] - 0s 42ms/step - loss: -0.2278 - acc: 0.8585
Epoch 36/50
1/1 [==============================] - 0s 39ms/step - loss: -0.2279 - acc: 0.8585
Epoch 37/50
1/1 [==============================] - 0s 41ms/step - loss: -0.2281 - acc: 0.8585
Epoch 38/50
1/1 [==============================] - 0s 38ms/step - loss: -0.2283 - acc: 0.8585
Epoch 39/50
1/1 [==============================] - 0s 40ms/step - loss: -0.2284 - acc: 0.8585
Epoch 40/50
1/1 [==============================] - 0s 36ms/step - loss: -0.2286 - acc: 0.8585
Epoch 41/50
1/1 [==============================] - 0s 44ms/step - loss: -0.2288 - acc: 0.8585
Epoch 42/50
1/1 [==============================] - 0s 47ms/step - loss: -0.2289 - acc: 0.8585
Epoch 43/50
1/1 [==============================] - 0s 39ms/step - loss: -0.2291 - acc: 0.8585
Epoch 44/50
1/1 [==============================] - 0s 38ms/step - loss: 

ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
ep 77: game finished, reward: -1.000000, epsilon: 1e-06
Epoch 1/50
1/1 [==============================] - 0s 66ms/step - loss: 0.0290 - acc: 0.8373
Epoch 2/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0286 - acc: 0.8373
Epoch 3/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0276 - acc: 0.8373
Epoch 4/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0263 - acc: 0.8373
Epoch 5/50
1/1 [==============================] - 0s 31ms/step - loss: 0.0246 - acc: 0.8373
Epoch 6/50
1/1 [====================

1/1 [==============================] - 0s 38ms/step - loss: -0.5574 - acc: 0.9501
Epoch 20/50
1/1 [==============================] - 0s 43ms/step - loss: -0.5585 - acc: 0.9501
Epoch 21/50
1/1 [==============================] - 0s 48ms/step - loss: -0.5596 - acc: 0.9501
Epoch 22/50
1/1 [==============================] - 0s 48ms/step - loss: -0.5608 - acc: 0.9501
Epoch 23/50
1/1 [==============================] - 0s 46ms/step - loss: -0.5619 - acc: 0.9501
Epoch 24/50
1/1 [==============================] - 0s 50ms/step - loss: -0.5631 - acc: 0.9501
Epoch 25/50
1/1 [==============================] - 0s 50ms/step - loss: -0.5643 - acc: 0.9501
Epoch 26/50
1/1 [==============================] - 0s 44ms/step - loss: -0.5655 - acc: 0.9501
Epoch 27/50
1/1 [==============================] - 0s 47ms/step - loss: -0.5667 - acc: 0.9501
Epoch 28/50
1/1 [==============================] - 0s 46ms/step - loss: -0.5679 - acc: 0.9501
Epoch 29/50
1/1 [==============================] - 0s 47ms/step - loss: 

1/1 [==============================] - 0s 31ms/step - loss: 0.2708 - acc: 0.8333
Epoch 44/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2681 - acc: 0.8333
Epoch 45/50
1/1 [==============================] - 0s 33ms/step - loss: 0.2655 - acc: 0.8333
Epoch 46/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2628 - acc: 0.8333
Epoch 47/50
1/1 [==============================] - 0s 33ms/step - loss: 0.2601 - acc: 0.8333
Epoch 48/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2575 - acc: 0.8333
Epoch 49/50
1/1 [==============================] - 0s 29ms/step - loss: 0.2548 - acc: 0.8333
Epoch 50/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2521 - acc: 0.8333
resetting env. episode reward total was -21.000000. running mean: -20.662379
ep 80: game finished, reward: -1.000000, epsilon: 1e-06
ep 80: game finished, reward: -1.000000, epsilon: 1e-06
ep 80: game finished, reward: -1.000000, epsilon: 1e-06
ep 80: game finished, r

KeyboardInterrupt: 

In [7]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [8]:
env.unwrapped.get_action_meanings()

# NOOP is the same as FIRE (standing still)
# LEFT is the same as LEFTFIRE (down)
# RIGHT is the same as RIGHTFIRE (up)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [9]:
env.unwrapped.get_keys_to_action()

{(): 0, (32,): 1, (100,): 2, (97,): 3, (32, 100): 4, (32, 97): 5}